In [111]:
# Import libraries
import requests, os
import urllib.request
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import statistics
import json


In [128]:
# Set the URL you want to webscrape from
all_games = []
for i in tqdm(range(1,50)):
    print(i)
    # Connect to the URL

    response = requests.get(f'https://boardgamegeek.com/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")

    for row in soup.find_all(id='row_'):
        all_games.append(row.find_all('a')[1].get('href'))
        







  0%|          | 0/74 [00:00<?, ?it/s]

1







  1%|▏         | 1/74 [00:00<00:15,  4.84it/s]




  3%|▎         | 2/74 [00:00<00:14,  4.87it/s]

2
3







  4%|▍         | 3/74 [00:00<00:14,  4.82it/s]

...






 96%|█████████▌| 71/74 [00:14<00:00,  4.75it/s]

72







 97%|█████████▋| 72/74 [00:15<00:00,  4.78it/s]




 99%|█████████▊| 73/74 [00:15<00:00,  4.92it/s]

73
74







100%|██████████| 74/74 [00:15<00:00,  4.79it/s]


In [129]:
print(len(all_games))
game_id = list(map(lambda x: x.split('/')[2], all_games))
l

7400
['/boardgame/174430/gloomhaven', '/boardgame/161936/pandemic-legacy-season-1', '/boardgame/167791/terraforming-mars', '/boardgame/182028/through-ages-new-story-civilization', '/boardgame/224517/brass-birmingham', '/boardgame/233078/twilight-imperium-fourth-edition', '/boardgame/12333/twilight-struggle', '/boardgame/187645/star-wars-rebellion', '/boardgame/220308/gaia-project', '/boardgame/169786/scythe']
['/boardgame/146652/legendary-encounters-alien-deck-building-game', '/boardgame/9609/war-ring', '/boardgame/148949/istanbul', '/boardgame/155821/inis', '/boardgame/155426/castles-mad-king-ludwig', '/boardgame/103343/game-thrones-board-game-second-edition', '/boardgame/14996/ticket-ride-europe', '/boardgame/233371/clank-space', '/boardgame/128882/resistance-avalon', '/boardgame/43111/chaos-old-world']


In [130]:
len(set(game_id))

7398

In [5]:
kid_games = []
for i in tqdm(range(1,15)):

    # Connect to the URL
    response = requests.get(f'https://boardgamegeek.com/childrensgames/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for row in soup.find_all(id='row_'):
        if row.find_all('td')[3].contents[0].strip()!='N/A':
            kid_games.append(row.find_all('a')[1].get('href'))



100%|██████████| 14/14 [00:03<00:00,  3.51it/s]


In [6]:
for elm in map(lambda x: x.split('/')[2], kid_games):
    game_id.append(elm)


In [7]:
family_games = []
for i in tqdm(range(1,16)):

    # Connect to the URL
    response = requests.get(f'https://boardgamegeek.com/familygames/browse/boardgame/page/{i}')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for row in soup.find_all(id='row_'):
        if row.find_all('td')[3].contents[0].strip()!='N/A':
            family_games.append(row.find_all('a')[1].get('href'))


100%|██████████| 15/15 [00:05<00:00,  2.84it/s]


In [8]:
for elm in map(lambda x: x.split('/')[2], family_games):
    game_id.append(elm)


In [9]:
len(set(game_id))

2340

In [10]:
game_id[0]

'174430'

In [131]:
import os
stats_kws = ['usersrated','average', 'bayesaverage', 'stddev', 'median', 'owned', 'trading','wanting','wishing','numcomments', 'numweights','averageweight']
info_kws = ['name', 'yearpublished', 'minplayers', 'maxplayers', 'playingtime','minplaytime','maxplaytime','age']
# See what's been saved
done = set()

for id in tqdm(set(game_id).difference(map(lambda x: x.split('.')[0],os.listdir('bbg_json')))):

    #for id in tqdm(set(game_id)):
    game_json = {}
    #First get the stats
    response = requests.get(f'https://www.boardgamegeek.com/xmlapi/boardgame/{id}?stats=1')

    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    for kw in info_kws:
        game_json[kw] = soup.find(kw).contents[0]
    game_json['description'] = soup.find('description').contents[0].replace('<br/>', ' ').strip()
    try:
        game_json['img_url'] = soup.find('thumbnail').contents[0]
    except IndexError:
        game_json['img_url'] = 'N/A'
    game_json['user_suggested_age'] = 0
    game_json['user_suggested_players'] = []

    for poll in soup.find_all('poll'):

        if poll.get('name') == "suggested_playerage":
            total_votes = float(poll.get('totalvotes'))
            if total_votes > 0:
                for elm in poll.find_all('result'):
                    game_json['user_suggested_age'] += float(elm.get('value').split(' ')[0])*float(elm.get('numvotes'))
                game_json['user_suggested_age'] /= total_votes
        elif poll.get('name') == 'suggested_numplayers':
            for results in poll.find_all('results'):
                suggested_num_dict = {'player_num' : results.get('numplayers')}
                for elm in results.find_all('result'):
                    suggested_num_dict[elm.get('value')] = elm.get('numvotes')
                game_json['user_suggested_players'].append(suggested_num_dict)
            
    stats = soup.find('ratings')
    game_json['stats'] = {}
    for kw in stats_kws:
        game_json['stats'][kw] = float(stats.find(kw).contents[0])
    num_comments = min(game_json['stats']['numcomments'],100)
    
                
    game_json['comments'] = []


    j = 1

    while (num_comments) > 0:
        response = requests.get(f'https://www.boardgamegeek.com/xmlapi/boardgame/{id}?comments=j')

        # Parse HTML and save to BeautifulSoup object¶
        soup = BeautifulSoup(response.text, "html.parser")
       
        
        for elm in soup.find_all('comment'):
            comment_dict = {}
            comment_dict['rating'] = elm.get('rating')
            comment_dict['username'] = elm.get('username')
            comment_dict['comment'] = elm.contents[0]
            game_json['comments'].append(comment_dict)
        j += 1
        num_comments -=50

    with open(f'bbg_json/{id}.json', 'w') as f:
        json.dump(game_json, f)







  0%|          | 0/5741 [00:00<?, ?it/s]




  0%|          | 1/5741 [00:00<31:27,  3.04it/s]




  0%|          | 2/5741 [00:00<33:09,  2.88it/s]




  0%|          | 3/5741 [00:01<51:27,  1.86it/s]




  0%|          | 4/5741 [00:02<50:14,  1.90it/s]




  0%|          | 5/5741 [00:02<49:23,  1.94it/s]




  0%|          | 6/5741 [00:03<52:02,  1.84it/s]




  0%|          | 7/5741 [00:03<49:51,  1.92it/s]




  0%|          | 8/5741 [00:04<45:32,  2.10it/s]




  0%|          | 9/5741 [00:04<39:14,  2.43it/s]




  0%|          | 10/5741 [00:04<40:40,  2.35it/s]




  0%|          | 11/5741 [00:05<41:58,  2.28it/s]




  0%|          | 12/5741 [00:05<41:06,  2.32it/s]




  0%|          | 13/5741 [00:06<45:39,  2.09it/s]




  0%|          | 14/5741 [00:06<39:31,  2.41it/s]




  0%|          | 15/5741 [00:07<40:55,  2.33it/s]




  0%|          | 16/5741 [00:07<38:23,  2.49it/s]




  0%|          | 17/5741 [00:07<36:36,  2.61it/s]




  0%|          | 18/5741 [00:08<38:48,  2

  2%|▏         | 118/5741 [00:51<40:02,  2.34it/s]




  2%|▏         | 119/5741 [00:52<35:19,  2.65it/s]




  2%|▏         | 120/5741 [00:52<36:35,  2.56it/s]




  2%|▏         | 121/5741 [00:52<36:12,  2.59it/s]




  2%|▏         | 122/5741 [00:53<34:50,  2.69it/s]




  2%|▏         | 123/5741 [00:53<36:27,  2.57it/s]




  2%|▏         | 124/5741 [00:54<36:57,  2.53it/s]




  2%|▏         | 125/5741 [00:54<38:07,  2.45it/s]




  2%|▏         | 126/5741 [00:55<38:22,  2.44it/s]




  2%|▏         | 127/5741 [00:55<46:38,  2.01it/s]




  2%|▏         | 128/5741 [00:56<42:40,  2.19it/s]




  2%|▏         | 129/5741 [00:56<39:56,  2.34it/s]




  2%|▏         | 130/5741 [00:56<38:02,  2.46it/s]




  2%|▏         | 131/5741 [00:57<36:29,  2.56it/s]




  2%|▏         | 132/5741 [00:57<46:28,  2.01it/s]




  2%|▏         | 133/5741 [00:58<42:35,  2.19it/s]




  2%|▏         | 134/5741 [00:58<42:00,  2.22it/s]




  2%|▏         | 135/5741 [00:59<41:16,  2.26it/s]




  2%|▏    

  4%|▍         | 235/5741 [01:41<34:44,  2.64it/s]




  4%|▍         | 236/5741 [01:41<31:59,  2.87it/s]




  4%|▍         | 237/5741 [01:42<33:04,  2.77it/s]




  4%|▍         | 238/5741 [01:42<31:42,  2.89it/s]




  4%|▍         | 239/5741 [01:42<32:21,  2.83it/s]




  4%|▍         | 240/5741 [01:43<33:26,  2.74it/s]




  4%|▍         | 241/5741 [01:43<38:27,  2.38it/s]




  4%|▍         | 242/5741 [01:44<43:26,  2.11it/s]




  4%|▍         | 243/5741 [01:44<42:00,  2.18it/s]




  4%|▍         | 244/5741 [01:45<53:11,  1.72it/s]




  4%|▍         | 245/5741 [01:46<1:00:15,  1.52it/s]




  4%|▍         | 246/5741 [01:47<52:04,  1.76it/s]  




  4%|▍         | 247/5741 [01:47<45:17,  2.02it/s]




  4%|▍         | 248/5741 [01:47<39:33,  2.31it/s]




  4%|▍         | 249/5741 [01:48<40:46,  2.24it/s]




  4%|▍         | 250/5741 [01:48<43:01,  2.13it/s]




  4%|▍         | 251/5741 [01:49<49:56,  1.83it/s]




  4%|▍         | 252/5741 [01:49<49:40,  1.84it/s]




  4%|▍

  6%|▌         | 351/5741 [02:39<37:59,  2.36it/s]




  6%|▌         | 352/5741 [02:39<38:54,  2.31it/s]




  6%|▌         | 353/5741 [02:40<38:39,  2.32it/s]




  6%|▌         | 354/5741 [02:40<38:22,  2.34it/s]




  6%|▌         | 355/5741 [02:41<41:03,  2.19it/s]




  6%|▌         | 356/5741 [02:41<35:18,  2.54it/s]




  6%|▌         | 357/5741 [02:41<31:39,  2.83it/s]




  6%|▌         | 358/5741 [02:42<32:57,  2.72it/s]




  6%|▋         | 359/5741 [02:42<35:10,  2.55it/s]




  6%|▋         | 360/5741 [02:43<34:18,  2.61it/s]




  6%|▋         | 361/5741 [02:43<31:53,  2.81it/s]




  6%|▋         | 362/5741 [02:43<32:50,  2.73it/s]




  6%|▋         | 363/5741 [02:44<40:02,  2.24it/s]




  6%|▋         | 364/5741 [02:45<46:10,  1.94it/s]




  6%|▋         | 365/5741 [02:45<42:18,  2.12it/s]




  6%|▋         | 366/5741 [02:45<41:00,  2.18it/s]




  6%|▋         | 367/5741 [02:46<39:02,  2.29it/s]




  6%|▋         | 368/5741 [02:46<38:04,  2.35it/s]




  6%|▋    

  8%|▊         | 468/5741 [03:30<45:21,  1.94it/s]




  8%|▊         | 469/5741 [03:31<42:41,  2.06it/s]




  8%|▊         | 470/5741 [03:31<40:22,  2.18it/s]




  8%|▊         | 471/5741 [03:32<52:57,  1.66it/s]




  8%|▊         | 472/5741 [03:32<47:24,  1.85it/s]




  8%|▊         | 473/5741 [03:33<42:38,  2.06it/s]




  8%|▊         | 474/5741 [03:33<40:37,  2.16it/s]




  8%|▊         | 475/5741 [03:34<45:47,  1.92it/s]




  8%|▊         | 476/5741 [03:34<42:18,  2.07it/s]




  8%|▊         | 477/5741 [03:35<40:39,  2.16it/s]




  8%|▊         | 478/5741 [03:35<38:22,  2.29it/s]




  8%|▊         | 479/5741 [03:35<36:55,  2.37it/s]




  8%|▊         | 480/5741 [03:36<35:41,  2.46it/s]




  8%|▊         | 481/5741 [03:36<35:51,  2.44it/s]




  8%|▊         | 482/5741 [03:37<36:06,  2.43it/s]




  8%|▊         | 483/5741 [03:37<42:07,  2.08it/s]




  8%|▊         | 484/5741 [03:38<39:03,  2.24it/s]




  8%|▊         | 485/5741 [03:38<37:33,  2.33it/s]




  8%|▊    

...

 48%|████▊     | 2777/5741 [21:22<19:35,  2.52it/s]




 48%|████▊     | 2778/5741 [21:23<18:53,  2.61it/s]




 48%|████▊     | 2779/5741 [21:23<16:40,  2.96it/s]




 48%|████▊     | 2780/5741 [21:23<17:50,  2.77it/s]




 48%|████▊     | 2781/5741 [21:24<15:58,  3.09it/s]




 48%|████▊     | 2782/5741 [21:24<18:21,  2.69it/s]




 48%|████▊     | 2783/5741 [21:25<18:43,  2.63it/s]




 48%|████▊     | 2784/5741 [21:25<19:39,  2.51it/s]




 49%|████▊     | 2785/5741 [21:26<26:11,  1.88it/s]




 49%|████▊     | 2786/5741 [21:26<24:38,  2.00it/s]




 49%|████▊     | 2787/5741 [21:27<25:30,  1.93it/s]




 49%|████▊     | 2788/5741 [21:27<23:38,  2.08it/s]




 49%|████▊     | 2789/5741 [21:27<20:06,  2.45it/s]




 49%|████▊     | 2790/5741 [21:28<19:43,  2.49it/s]




 49%|████▊     | 2791/5741 [21:28<18:51,  2.61it/s]




 49%|████▊     | 2792/5741 [21:29<20:03,  2.45it/s]




 49%|████▊     | 2793/5741 [21:29<20:51,  2.36it/s]




 49%|████▊     | 2794/5741 [21:29<20:20,  2.41it

 50%|█████     | 2892/5741 [22:12<21:46,  2.18it/s]




 50%|█████     | 2893/5741 [22:13<21:30,  2.21it/s]




 50%|█████     | 2894/5741 [22:13<20:50,  2.28it/s]




 50%|█████     | 2895/5741 [22:14<19:55,  2.38it/s]




 50%|█████     | 2896/5741 [22:14<19:48,  2.39it/s]




 50%|█████     | 2897/5741 [22:15<21:17,  2.23it/s]




 50%|█████     | 2898/5741 [22:15<21:36,  2.19it/s]




 50%|█████     | 2899/5741 [22:15<20:23,  2.32it/s]




 51%|█████     | 2900/5741 [22:16<19:42,  2.40it/s]




 51%|█████     | 2901/5741 [22:16<18:22,  2.58it/s]




 51%|█████     | 2902/5741 [22:17<20:32,  2.30it/s]




 51%|█████     | 2903/5741 [22:18<32:34,  1.45it/s]




 51%|█████     | 2904/5741 [22:18<29:34,  1.60it/s]




 51%|█████     | 2905/5741 [22:19<29:21,  1.61it/s]




 51%|█████     | 2906/5741 [22:19<26:42,  1.77it/s]




 51%|█████     | 2907/5741 [22:20<24:41,  1.91it/s]




 51%|█████     | 2908/5741 [22:20<24:05,  1.96it/s]




 51%|█████     | 2909/5741 [22:21<22:47,  2.07it

 52%|█████▏    | 3007/5741 [23:05<16:32,  2.75it/s]




 52%|█████▏    | 3008/5741 [23:05<20:06,  2.26it/s]




 52%|█████▏    | 3009/5741 [23:06<20:57,  2.17it/s]




 52%|█████▏    | 3010/5741 [23:06<18:03,  2.52it/s]




 52%|█████▏    | 3011/5741 [23:06<17:43,  2.57it/s]




 52%|█████▏    | 3012/5741 [23:07<17:48,  2.55it/s]




 52%|█████▏    | 3013/5741 [23:07<22:34,  2.01it/s]




 52%|█████▏    | 3014/5741 [23:08<20:50,  2.18it/s]




 53%|█████▎    | 3015/5741 [23:08<19:36,  2.32it/s]




 53%|█████▎    | 3016/5741 [23:09<18:53,  2.40it/s]




 53%|█████▎    | 3017/5741 [23:09<19:05,  2.38it/s]




 53%|█████▎    | 3018/5741 [23:09<18:47,  2.41it/s]




 53%|█████▎    | 3019/5741 [23:10<18:44,  2.42it/s]




 53%|█████▎    | 3020/5741 [23:10<19:25,  2.33it/s]




 53%|█████▎    | 3021/5741 [23:11<19:35,  2.31it/s]




 53%|█████▎    | 3022/5741 [23:12<27:37,  1.64it/s]




 53%|█████▎    | 3023/5741 [23:12<22:18,  2.03it/s]




 53%|█████▎    | 3024/5741 [23:12<21:38,  2.09it

 54%|█████▍    | 3122/5741 [23:57<17:37,  2.48it/s]




 54%|█████▍    | 3123/5741 [23:57<15:36,  2.80it/s]




 54%|█████▍    | 3124/5741 [23:58<16:08,  2.70it/s]




 54%|█████▍    | 3125/5741 [23:58<16:15,  2.68it/s]




 54%|█████▍    | 3126/5741 [23:58<16:12,  2.69it/s]




 54%|█████▍    | 3127/5741 [23:59<15:33,  2.80it/s]




 54%|█████▍    | 3128/5741 [23:59<15:41,  2.78it/s]




 55%|█████▍    | 3129/5741 [23:59<17:56,  2.43it/s]




 55%|█████▍    | 3130/5741 [24:00<17:26,  2.49it/s]




 55%|█████▍    | 3131/5741 [24:00<17:38,  2.46it/s]




 55%|█████▍    | 3132/5741 [24:01<20:49,  2.09it/s]




 55%|█████▍    | 3133/5741 [24:01<20:51,  2.08it/s]




 55%|█████▍    | 3134/5741 [24:02<22:56,  1.89it/s]




 55%|█████▍    | 3135/5741 [24:02<20:20,  2.13it/s]




 55%|█████▍    | 3136/5741 [24:03<19:16,  2.25it/s]




 55%|█████▍    | 3137/5741 [24:03<17:52,  2.43it/s]




 55%|█████▍    | 3138/5741 [24:03<16:57,  2.56it/s]

KeyboardInterrupt: 

In [104]:
len(os.listdir('bbg_json')), len(set(game_id))

(2340, 100)